---

# **Documento Técnico: Algoritmo de Roteamento Semântico com Base em Similaridade de Vetores**

## **Visão Geral**

Este documento descreve o processo de implementação de um "Roteador Semântico", um algoritmo que usa embeddings (representações vetoriais) para rotear uma solicitação para o serviço ou endpoint mais apropriado, com base na similaridade semântica da consulta de entrada.

### **Descrição do Algoritmo**

O algoritmo segue os seguintes passos:

1. **Gerar Embedding da Consulta**: A consulta é convertida em um embedding (representação vetorial) usando um modelo de embeddings.
2. **Buscar Similaridades**: O embedding da consulta é comparado com um conjunto de embeddings pré-existentes no banco de dados para encontrar as correspondências mais próximas.
3. **Agrupar por Rota**: As correspondências mais próximas são agrupadas pelas rotas ou serviços a que pertencem.
4. **Filtrar Pontuações**: O algoritmo filtra as rotas que não atendem a um determinado limite, com base na soma de suas pontuações de similaridade.
5. **Selecionar a Melhor Rota**: Por fim, a rota com a maior pontuação de similaridade é escolhida para lidar com a consulta.




> Modulos Nativos ou instalados

In [ ]:
import uuid
from typing import Dict, List, Literal


from sqlmodel import Session, select

> User Modules

In [ ]:
from database import pg_engine # SQLAlchemy engine
from models import Embedding, Collection # SQLModel models
from utils import embed # Embedding function

In [ ]:
def _set_collection_id(session: Session, layer: Literal['entry_layer', 'customer_layer']) -> uuid.UUID:
    # Busca os IDs das coleções com base na camada
    collections = session.exec(
        select(Collection)
    ).all()
    
    collection_ids = {collection.name: collection.uuid for collection in collections}

    if layer == 'entry_layer':
        return collection_ids['routes']
    elif layer == 'customer_layer':
        return collection_ids['customer_routes']

### **Passos do Algoritmo**

#### 1. **Gerar Embedding da Consulta**

In [ ]:
query = "create a new customer in ge3"
layer = "entry_layer"
threshold = 0.5

vector = embed(query)

- A consulta, "create a new customer in ge3", é convertida em uma representação vetorial usando um modelo de embeddings (por exemplo, GPT-3 da OpenAI ou modelos semelhantes).
- A variável `threshold` representa a pontuação mínima de similaridade exigida para que uma rota seja considerada.

#### 2. **Busca de Similaridade**
- Os embeddings dos documentos ou serviços predefinidos estão armazenados no modelo `Embedding`.

In [ ]:
with Session(pg_engine) as session:
    collection_id = _set_collection_id(session, layer)

    distance_score = Embedding.embedding.max_inner_product(vector).label('distance_score')

    routes = session.exec(
        select(-distance_score, Embedding.document, Embedding.cmetadata)
        .where(collection_id == Embedding.collection_id)
        .order_by(distance_score)
        .limit(25)
    ).all()

- O embedding da consulta é comparado com esses embeddings armazenados usando um produto interno (produto escalar) para encontrar as 25 rotas (documentos) mais semelhantes no banco de dados.
- No cálculo de similaridade, usamos o produto interno multiplicado por -1, pois o Postgres suporta apenas varreduras de índice de ordem ASC em operadores, assim o valor vem negativo da consulta SQL.


In [ ]:
with Session(pg_engine) as session:
    collection_id = _set_collection_id(session, layer)

    distance_score = Embedding.embedding.max_inner_product(vector).label('distance_score')
    
    routes = session.exec(
        select((1 - distance_score / 2), Embedding.document, Embedding.cmetadata)  # Ajuste com cosseno
        .where(collection_id == Embedding.collection_id)
        .order_by(distance_score)
        .limit(25)
    ).all()

- No cálculo de similaridade por cosseno, ajustamos para 1 - (distance_score / 2) para garantir que os valores resultantes estejam entre 0 e 1.
- onde 0 significa "completamente diferente" e 1 significa "idêntico".

#### 3. **Agrupamento por Rota**

In [ ]:
grouped_data: Dict[str, List[float]] = {}
for score, document, metadata in routes:
    route_name = metadata['route_name']
    if route_name not in grouped_data:
        grouped_data[route_name] = []
    grouped_data[route_name].append(score)

- As pontuações de similaridade são agrupadas pelo nome da rota ou serviço ao qual pertencem, criando um dicionário onde a chave é o nome da rota e o valor é uma lista de pontuações de similaridade.

#### 4. **Filtragem por Limite**


In [ ]:
filtered_routes = {route_name: scores for route_name, scores in grouped_data.items() if sum(scores) >= threshold}

- As rotas são filtradas com base em um limite de pontuação. Somente as rotas com uma pontuação total de similaridade maior ou igual ao limite são mantidas.

#### 5. **Seleção da Melhor Rota**


In [ ]:
sum_scores = {route_name: sum(scores) for route_name, scores in filtered_routes.items()}

```shell
{'customers': 15.390514, 'automations': 7.2249}
```

In [ ]:
sorted_routes = dict(sorted(sum_scores.items(), key=lambda item: item[1], reverse=True))

best_route = max(sorted_routes, key=sorted_routes.get)

```shell
'customer'
```
- As rotas são classificadas pela pontuação total de similaridade em ordem decrescente, e a rota com a maior pontuação é selecionada como a melhor rota para a consulta.

---

## **Implementação em C#**

Esta seção fornece orientações para a implementação do algoritmo em C#. Os passos principais incluem a geração de embeddings, a consulta ao banco de dados para recuperar embeddings e a filtragem das rotas com base nas pontuações de similaridade.

### **Pré-requisitos**
- **Geração de Embeddings**: Utilize um modelo ou biblioteca de NLP existente, como HuggingFace Transformers ou OpenAI, para gerar embeddings vetoriais para consultas no seu aplicativo C#. Você pode usar uma API ou um modelo, como o GPT-3, para converter consultas em vetores.
- **Configuração do Banco de Dados**: Armazene os embeddings de cada documento/rota em um banco de dados. Você pode usar PostgreSQL ou qualquer outro banco relacional, juntamente com um ORM em C#, como o Entity Framework.

### **Componentes Principais**

#### 1. **Gerar Embedding da Consulta**

```csharp
// Supondo que você tenha um serviço que gera embeddings para textos
var embeddingService = new EmbeddingService();
string query = "create a new customer in ge3";
float[] queryVector = embeddingService.Embed(query);
```

#### 2. **Buscar Pontuações de Similaridade**

```csharp
using (var context = new EmbeddingContext())
{
    var collectionId = SetCollectionId("entry_layer");

    // Obter as 25 rotas mais semelhantes com base no produto interno
    var routes = context.Embeddings
        .Where(e => e.CollectionId == collectionId)
        .Select(e => new {
            DistanceScore = MaxInnerProduct(queryVector, e.Vector),
            RouteName = e.RouteName,
            Metadata = e.Metadata
        })
        .OrderByDescending(e => e.DistanceScore)
        .Take(25)
        .ToList();
}
```
- Você precisará implementar a função de similaridade ou [#entity-framework-core](https://github.com/pgvector/pgvector-dotnet?tab=readme-ov-file#entity-framework-core)

#### 3. **Agrupar por Rota**

```csharp
var groupedData = new Dictionary<string, List<float>>();

foreach (var route in routes)
{
    if (!groupedData.ContainsKey(route.RouteName))
    {
        groupedData[route.RouteName] = new List<float>();
    }
    groupedData[route.RouteName].Add(route.DistanceScore);
}
```

#### 4. **Filtrar por Limite**

```csharp
var threshold = 0.5;
var filteredRoutes = groupedData
    .Where(g => g.Value.Sum() >= threshold)
    .ToDictionary(g => g.Key, g => g.Value);
```

#### 5. **Selecionar a Melhor Rota**

```csharp
var bestRoute = filteredRoutes
    .OrderByDescending(g => g.Value.Sum())
    .FirstOrDefault().Key;

Console.WriteLine($"A melhor rota para a consulta é: {bestRoute}");
```

### **Design de Banco de Dados para Embeddings**
- **Tabela de Embeddings**:
    - `Id`: Chave primária
    - `RouteName`: Nome da rota (ex: "customers", "automations")
    - `Vector`: O vetor do embedding, armazenado como `float[]` ou string serializada.
    - `Metadata`: Metadados adicionais relacionados à rota
    - `CollectionId`: Identifica a camada ou coleção a que o embedding pertence (ex: "entry_layer")

---

## **Conclusão**

Este documento descreve o processo de construção de um roteador semântico tanto em Python (implementação original) quanto como seria idealmente em C#. Os principais passos envolvem a geração de embeddings para a consulta, o cálculo de pontuações de similaridade com embeddings armazenados, o agrupamento dos resultados por rota e a seleção da rota mais relevante com base nas pontuações.

Seguindo as orientações acima, um desenvolvedor pode implementar um algoritmo de roteamento escalável e eficiente com base em similaridade vetorial em um ambiente C#.

--- 
